In [1]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from torch.autograd import Variable
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
import torch.optim as optim

In [3]:
train_data= datasets.MNIST(root='./data',train=True,transform=transforms.ToTensor())
test_data= datasets.MNIST(root='./data',train=False,transform=transforms.ToTensor())

In [4]:
train_load=torch.utils.data.DataLoader(dataset=train_data,batch_size=1000,shuffle=True)
test_load=torch.utils.data.DataLoader(dataset=test_data,batch_size=1000,shuffle=True)

In [5]:
class node(nn.Module):
    def __init__(self):
        super(node, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
model = node()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.02)

In [7]:
model=model.cuda()

In [8]:
for epoch in range(5):
    model.train()
    running_loss = 0.0
    for i,(images, labels) in enumerate(train_load):
        images=images.cuda()
        labels=labels.cuda()
        outputs = model(images)
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        print(f" for loop {i} loss is {loss}")

    print(f"Epoch [{epoch}], Loss: {running_loss/len(train_load):.4f}")

 for loop 0 loss is 2.3049581050872803
 for loop 1 loss is 6.0979838371276855
 for loop 2 loss is 2.6887102127075195
 for loop 3 loss is 2.2874104976654053
 for loop 4 loss is 2.296189069747925
 for loop 5 loss is 2.2817933559417725
 for loop 6 loss is 2.244628429412842
 for loop 7 loss is 2.168532371520996
 for loop 8 loss is 2.0762507915496826
 for loop 9 loss is 1.8418183326721191
 for loop 10 loss is 1.5740128755569458
 for loop 11 loss is 1.2199742794036865
 for loop 12 loss is 0.887757420539856
 for loop 13 loss is 0.7391120195388794
 for loop 14 loss is 0.592476487159729
 for loop 15 loss is 0.5854935050010681
 for loop 16 loss is 0.6191180348396301
 for loop 17 loss is 0.5231968760490417
 for loop 18 loss is 0.5314329266548157
 for loop 19 loss is 0.5261539220809937
 for loop 20 loss is 0.4870451092720032
 for loop 21 loss is 0.4626319110393524
 for loop 22 loss is 0.3893029987812042
 for loop 23 loss is 0.3183366358280182
 for loop 24 loss is 0.3808688223361969
 for loop 25 lo

In [10]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_load:
        images=images.cuda()
        labels=labels.cuda()

        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")

Accuracy on test set: 98.48%
